[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs/tree/develop)](https://github.com/Mearman/openalex-docs/tree/develop)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/develop/api-entities/concepts/README.ipynb)](https://github.com/Mearman/openalex-docs/blob/develop/api-entities/concepts/README.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/develop/api-entities/concepts/README.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/develop/api-entities/concepts/README.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# 💡 Concepts

Concepts are abstract ideas that works are about. OpenAlex indexes about 65k concepts.

* Get all the concepts used by OpenAlex:
  [`https://api.openalex.org/concepts`](https://api.openalex.org/concepts)&#x20;

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/concepts


response = concepts_api.get_concepts(
	
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

The [Canonical External ID](./../../how-to-use-the-api/get-single-entities/README.md#canonical-external-ids) for OpenAlex concepts is the Wikidata ID, and each of our concepts has one, because all OpenAlex concepts are also Wikidata concepts.

Concepts are hierarchical, like a tree. There are 19 root-level concepts, and six layers of descendants branching out from them, containing about 65 thousand concepts all told. This concept tree is a modified version of [the one created by MAG](https://arxiv.org/abs/1805.12216).&#x20;

You can view all the concepts and their position in the tree [as a spreadsheet here](https://docs.google.com/spreadsheets/d/1LBFHjPt4rj_9r0t0TTAlT68NwOtNH8Z21lBMsJDMoZg/edit#gid=1473310811). About 85% of works are tagged with at least one concept (here's the [breakdown of concept counts per work](https://docs.google.com/spreadsheets/d/17DoJjyl1XVNZdVWs7fUy91z69U2tD8qtnBsaqJ-Zigo/edit#gid=0)).

## How concepts are assigned

Each work is tagged with multiple concepts, based on the title, abstract, and the title of its host venue. The tagging is done using an automated classifier that was trained on MAG’s corpus; you can read more about the development and operation of this classifier in [Automated concept tagging for OpenAlex, an open index of scholarly articles.](https://docs.google.com/document/d/1OgXSLriHO3Ekz0OYoaoP_h0sPcuvV4EqX7VgLLblKe4/edit) You can implement the classifier yourself using [our models and code](https://github.com/ourresearch/openalex-concept-tagging).

A score is available for each [concept in a work](./../works/work-object/README.md#concepts), showing the classifier's confidence in choosing that concept. However, when assigning a lower-level child concept, we also assign all of its parent concepts all the way up to the root. This means that some concept assignment scores will be 0.0. The tagger adds concepts to works written in different languages, but it is optimized for English.

Concepts are linked to works via the [`concepts`](./../works/work-object/README.md#concepts) property. They’re also linked to [authors](./../authors/author-object.ipynb), [sources](./../sources/sources-object.md), and [institutions](./../institutions/institution-object.ipynb) via the `x_concepts` property, and to other concepts via the [`ancestors`](./concept-object.md#ancestors) and [`related_concepts`](./concept-object.md#related_concepts) properties.

## What's next

Learn more about what you can do with concepts:

* [The Concept object](./concept-object.ipynb)
* [Get a single concept](./get-a-single-concept.ipynb)
* [Get lists of concepts](./get-lists-of-concepts.ipynb)
* [Filter concepts](./filter-concepts.ipynb)
* [Search concepts](./search-concepts.ipynb)
* [Group concepts](./group-concepts.ipynb)